In [81]:
import agents
import game
import mechanisms

In [82]:
import importlib
importlib.reload(agents)
importlib.reload(game)
importlib.reload(mechanisms)

<module 'mechanisms' from 'C:\\Users\\billz\\PycharmProjects\\CS_281\\Finding_Friends\\mechanisms.py'>

In [87]:
NUM_PLAYERS = 5
LVL_CAP = 100
WIN_PROB = 0.4

ff_agents = [agents.Basic_Agent for i in range(NUM_PLAYERS - 2)]
ff_agents.append(agents.Lowest_Level_Agent)
ff_agents.append(agents.Lowest_Level_Agent)

ff_game = game.Game(players=ff_agents, logging_level='INFO')

ff_mech = mechanisms.Baseline_Mechanism(p=WIN_PROB)

In [88]:
for i in range(20):
    ff_game.play(cap=LVL_CAP, mechanism=ff_mech)
    ff_game.reset()

In [24]:
type(ff_mech).__name__

'Baseline_Mechanism'